# Amazon Forecast: predicting time-series at scale

Forecasting is used in a variety of applications and business use cases: For example, retailers need to forecast the sales of their products to decide how much stock they need by location, Manufacturers need to estimate the number of parts required at their factories to optimize their supply chain, Businesses need to estimate their flexible workforce needs, Utilities need to forecast electricity consumption needs in order to attain an efficient energy network, and 
enterprises need to estimate their cloud infrastructure needs.

<img src="../../common/images/amazon_forecast.png">


# Table of Contents

* Step 0: [Setting up](#setup)
* Step 1: [Preparing the Datasets](#prepare)
* Step 2: [Importing the Data](#import)
 * Step 2a: [Creating a Dataset Group](#create)
 * Step 2b: [Creating a Target Dataset](#target)
 * Step 2c: [Creating a Related Dataset](#related)
 * Step 2d: [Update the Dataset Group](#update)
 * Step 2e: [Creating a Target Time Series Dataset Import Job](#targetImport)
 * Step 2f: [Creating a Related Time Series Dataset Import Job](#relatedImport)
* Step 3: [Choosing an Algorithm and Evaluating its Performance](#algo)
 * Step 3a: [Choosing DeepAR+](#DeepAR)
 * Step 3b: [Choosing Prophet](#prophet)
* Step 4: [Computing Error Metrics from Backtesting](#error)
* Step 5: [Creating a Forecast](#forecast)
* Step 6: [Querying the Forecasts](#query)
* Step 7: [Exporting the Forecasts](#export)
* Step 8: [Clearning up your Resources](#cleanup)


#  First let us setup Amazon Forecast<a class="anchor" id="setup">

This section sets up the permissions and relevant endpoints.

In [ ]:
import sys
import os

import boto3
import pandas as pd
import matplotlib.pyplot as plt

# importing forecast notebook utility from notebooks/common directory
sys.path.insert( 0, os.path.abspath("../../common") )
import util

plt.rcParams['figure.figsize'] = (15.0, 5.0)

Configure the S3 bucket name and region name for this lesson.

- If you don't have an S3 bucket, create it first on S3.
- Although we have set the region to us-west-2 as a default value below, you can choose any of the regions that the service is available in.

In [ ]:
text_widget_bucket = util.create_text_widget( "bucket_name", "input your S3 bucket name" )
text_widget_region = util.create_text_widget( "region", "input region name.", default_value="us-west-2" )

In [ ]:
bucket_name = text_widget_bucket.value
assert bucket_name, "bucket_name not set."

region = text_widget_region.value
assert region, "region not set."

In [ ]:
session = boto3.Session(region_name=region) 
forecast = session.client(service_name='forecast') 
forecast_query = session.client(service_name='forecastquery')

In [ ]:
# Create the role to provide to Amazon Forecast.
role_name = "ForecastNotebookRole-RelatedTimeSeries"
role_arn = util.get_or_create_iam_role( role_name = role_name )

# Overview

<img src="images/outline.png">

<img src="../../common/images/forecast_workflow.png">

The above figure summarizes the key workflow of using Forecast. 

# Step 1: Preparing the Datasets<a class="anchor" id="prepare">

In [ ]:
bike_df = pd.read_csv("data/train.csv", dtype = object)
bike_df.head()

In [ ]:
bike_df['count'] = bike_df['count'].astype('float')
bike_df['workingday'] = bike_df['workingday'].astype('float')

We take about two and a half week's of hourly data for demonstration, just for the purpose that there's no missing data in the whole range.

In [ ]:
bike_df_small = bike_df[-2*7*24-24*3:].copy()
bike_df_small['item_id'] = "bike_12"

Let us plot the time series first.

In [ ]:
bike_df_small.plot(x='datetime', y='count', figsize=(15, 8))

We can see that the target time series seem to have a drop over weekends. Next let's plot both the target time series and the related time series that indicates whether today is a `workday` or not. More precisely, $r_t = 1$ if $t$ is a work day and 0 if not.

In [ ]:
plt.figure(figsize=(15, 8))
ax = plt.gca()
bike_df_small.plot(x='datetime', y='count', ax=ax);
ax2 = ax.twinx()
bike_df_small.plot(x='datetime', y='workingday', color='red', ax=ax2);

Notice that to use the related time series, we need to ensure that the related time series covers the whole target time series, as well as the future values as specified by the forecast horizon. More precisely, we need to make sure:
```
len(related time series) >= len(target time series) + forecast horizon
```
Basically, all items need to have data start at or before the item start date, and have data until the forecast horizon (i.e. the latest end date across all items + forecast horizon).  Additionally, there should be no missing values in the related time series. The following picture illustrates the desired logic. 

<img src="images/rts_viz.png">

For more details regarding how to prepare your Related Time Series dataset, please refer to the public documentation <a href="https://docs.aws.amazon.com/forecast/latest/dg/related-time-series-datasets.html">here</a>. 

Suppose in this particular example, we wish to forecast for the next 24 hours, and thus we generate the following dataset.

In [ ]:
target_df = bike_df_small[['item_id', 'datetime', 'count']][:-24]
rts_df = bike_df_small[['item_id', 'datetime', 'workingday']]

In [ ]:
target_df.head(5)

As we can see, the length of the related time series is equal to the length of the target time series plus the forecast horizon. 

In [ ]:
print(len(target_df), len(rts_df))
assert len(target_df) + 24 == len(rts_df), "length doesn't match"

Next we check whether there are "holes" in the related time series.  

In [ ]:
assert len(rts_df) == len(pd.date_range(
    start=list(rts_df['datetime'])[0],
    end=list(rts_df['datetime'])[-1],
    freq='H'
)), "missing entries in the related time series"

Everything looks fine, and we plot both time series again. As it can be seen, the related time series (indicator of whether the current day is a workday or not) is longer than the target time series.  The binary working day indicator feature is a good example of a related time series, since it is known at all future time points.  Other examples of related time series include holiday and promotion features.

In [ ]:
plt.figure(figsize=(15, 10))
ax = plt.gca()
target_df.plot(x='datetime', y='count', ax=ax);
ax2 = ax.twinx()
rts_df.plot(x='datetime', y='workingday', color='red', ax=ax2);

In [ ]:
target_df.to_csv("data/bike_small.csv", index= False, header = False)
rts_df.to_csv("data/bike_small_rts.csv", index= False, header = False)

In [ ]:
s3 = session.client('s3')

In [ ]:
key = "bike_small"

s3.upload_file(Filename="data/bike_small.csv", Bucket = bucket_name, Key = f"{key}/bike.csv")
s3.upload_file(Filename="data/bike_small_rts.csv", Bucket = bucket_name, Key = f"{key}/bike_rts.csv")

# Step 2. Importing the Data<a class="anchor" id="import">


Now we are ready to import the datasets into the Forecast service. Starting from the raw data, Amazon Forecast automatically extracts the dataset that is suitable for forecasting. As an example, a retailer normally records the transaction record such as
<img src="images/data_format.png">
<img src="images/timestamp.png">



In [ ]:
project = "bike_rts_demo"
idx = 4

In [ ]:
s3_data_path = f"s3://{bucket_name}/{key}"

Below, we specify key input data and forecast parameters

In [ ]:
freq = "H"
forecast_horizon = 24
timestamp_format = "yyyy-MM-dd HH:mm:ss"
delimiter = ','

## Step 2a. Creating a Dataset Group<a class="anchor" id="create">
First let's create a dataset group and then update it later to add our datasets.

In [ ]:
dataset_group = f"{project}_gp_{idx}"
dataset_arns = []
create_dataset_group_response = forecast.create_dataset_group(Domain="RETAIL",
                                                          DatasetGroupName=dataset_group,
                                                          DatasetArns=dataset_arns)

In [ ]:
print(f'Creating dataset group {dataset_group}')

In [ ]:
dataset_group_arn = create_dataset_group_response['DatasetGroupArn']

In [ ]:
forecast.describe_dataset_group(DatasetGroupArn=dataset_group_arn)

## Step 2b. Creating a Target Dataset<a class="anchor" id="target">
In this example, we will define a target time series. This is a required dataset to use the service.

Below we specify the target time series name af_demo_ts_4.

In [ ]:
ts_dataset_name = f"{project}_ts_{idx}"
print(ts_dataset_name)

Next, we specify the schema of our dataset below. Make sure the order of the attributes (columns) matches the raw data in the files. We follow the same three attribute format as the above example.

In [ ]:
ts_schema_val = [{"AttributeName": "item_id", "AttributeType": "string"},
              {"AttributeName": "timestamp", "AttributeType": "timestamp"},
              {"AttributeName": "demand", "AttributeType": "float"}]
ts_schema = {"Attributes": ts_schema_val}

In [ ]:
print(f'Creating target dataset {ts_dataset_name}')

In [ ]:
response = forecast.create_dataset(Domain="RETAIL",
                               DatasetType='TARGET_TIME_SERIES',
                               DatasetName=ts_dataset_name,
                               DataFrequency=freq,
                               Schema=ts_schema
                              )

In [ ]:
ts_dataset_arn = response['DatasetArn']

In [ ]:
forecast.describe_dataset(DatasetArn=ts_dataset_arn)

## Step 2c. Creating a Related Dataset<a class="anchor" id="related">
In this example, we will define a related time series.

Specify the related time series name af_demo_rts_4.

In [ ]:
rts_dataset_name = f"{project}_rts_{idx}"
print(rts_dataset_name)

Specify the schema of your dataset here. Make sure the order of columns matches the raw data files. We follow the same three column format as the above example.

In [ ]:
rts_schema_val = [{"AttributeName": "item_id", "AttributeType": "string"},
              {"AttributeName": "timestamp", "AttributeType": "timestamp"},
              {"AttributeName": "price", "AttributeType": "float"}]
rts_schema = {"Attributes": rts_schema_val}

In [ ]:
print(f'Creating related dataset {rts_dataset_name}')

In [ ]:
response = forecast.create_dataset(Domain="RETAIL",
                               DatasetType='RELATED_TIME_SERIES',
                               DatasetName=rts_dataset_name,
                               DataFrequency=freq,
                               Schema=rts_schema
                              )

In [ ]:
rts_dataset_arn = response['DatasetArn']

In [ ]:
forecast.describe_dataset(DatasetArn=rts_dataset_arn)

## Step 2d. Updating the dataset group with the datasets we created<a class="anchor" id="update">
You can have multiple datasets under the same dataset group. Update it with the datasets we created before.

In [ ]:
dataset_arns = []
dataset_arns.append(ts_dataset_arn)
dataset_arns.append(rts_dataset_arn)
forecast.update_dataset_group(DatasetGroupArn=dataset_group_arn, DatasetArns=dataset_arns)

In [ ]:
forecast.describe_dataset_group(DatasetGroupArn=dataset_group_arn)

## Step 2e. Creating a Target Time Series Dataset Import Job<a class="anchor" id="targetImport">

In [ ]:
ts_s3_data_path = f"{s3_data_path}/bike.csv"

In [ ]:
ts_dataset_import_job_response = forecast.create_dataset_import_job(DatasetImportJobName=dataset_group,
                                                             DatasetArn=ts_dataset_arn,
                                                             DataSource= {
                                                                 "S3Config" : {
                                                                     "Path": ts_s3_data_path,
                                                                     "RoleArn": role_arn
                                                                 } 
                                                             },
                                                             TimestampFormat=timestamp_format)

In [ ]:
ts_dataset_import_job_arn=ts_dataset_import_job_response['DatasetImportJobArn']

In [ ]:
status = util.wait(lambda: forecast.describe_dataset_import_job(DatasetImportJobArn=ts_dataset_import_job_arn))
assert status

## Step 2f. Creating a Related Time Series Dataset Import Job<a class="anchor" id="relatedImport">

In [ ]:
rts_s3_data_path = f"{s3_data_path}/bike_rts.csv"

In [ ]:
rts_dataset_import_job_response = forecast.create_dataset_import_job(DatasetImportJobName=dataset_group,
                                                             DatasetArn=rts_dataset_arn,
                                                             DataSource= {
                                                                 "S3Config" : {
                                                                     "Path": rts_s3_data_path,
                                                                     "RoleArn": role_arn
                                                                 } 
                                                             },
                                                             TimestampFormat=timestamp_format)

In [ ]:
rts_dataset_import_job_arn=rts_dataset_import_job_response['DatasetImportJobArn']

In [ ]:
status = util.wait(lambda: forecast.describe_dataset_import_job(DatasetImportJobArn=rts_dataset_import_job_arn))
assert status

# Step 3. Choosing an algorithm and evaluating its performance<a class="anchor" id="algo">

Once the datasets are specified with the corresponding schema, Amazon Forecast will automatically aggregate all the relevant pieces of information for each item, such as sales, price, promotions, as well as categorical attributes, and generate the desired dataset. Next, one can choose an algorithm (forecasting model) and evaluate how well this particular algorithm works on this dataset. The following graph gives a high-level overview of the forecasting models.
<img src="images/recipes.png">
<img src="images/pred_details.png">


Amazon Forecast provides several state-of-the-art forecasting algorithms including classic forecasting methods such as ETS, ARIMA, Prophet and deep learning approaches such as DeepAR+. Classical forecasting methods, such as Autoregressive Integrated Moving Average (ARIMA) or Exponential Smoothing (ETS), fit a single model to each individual time series, and then use that model to extrapolate the time series into the future. Amazon's Non-Parametric Time Series (NPTS) forecaster also fits a single model to each individual time series.  Unlike the naive or seasonal naive forecasters that use a fixed time index (the previous index $T-1$ or the past season $T - \tau$) as the prediction for time step $T$, NPTS randomly samples a time index $t \in \{0, \dots T-1\}$ in the past to generate a sample for the current time step $T$.

In many applications, you may encounter many similar time series across a set of cross-sectional units. Examples of such time series groupings are demand for different products, server loads, and requests for web pages. In this case, it can be beneficial to train a single model jointly over all of these time series. DeepAR+ takes this approach, outperforming the standard ARIMA and ETS methods when your dataset contains hundreds of related time series. The trained model can also be used for generating forecasts for new time series that are similar to the ones it has been trained on. While deep learning approaches can outperform standard methods, this is only possible when there is sufficient data available for training. It is not true for example when one trains a neural network with a time-series contains only a few dozens of observations. Amazon Forecast provides the best of two worlds allowing users to either choose a specific algorithm or let Amazon Forecast automatically perform model selection. 

## How to evaluate a forecasting model?

Before moving forward, let's first introduce the notion of *backtest* when evaluating forecasting models. The key difference between evaluating forecasting algorithms and standard ML applications is that we need to make sure there is no future information gets used in the past. In other words, the procedure needs to be causal. 

<img src="images/backtest.png">


In this notebook, let's compare the neural network based method, DeepAR+ with Facebook's open-source Bayesian method Prophet. 


In [ ]:
algorithm_arn = 'arn:aws:forecast:::algorithm/'

## Step 3a. Choosing DeepAR+<a class="anchor" id="DeepAR">

In [ ]:
algorithm = 'Deep_AR_Plus'
algorithm_arn_deep_ar_plus = algorithm_arn + algorithm
predictor_name_deep_ar = f'{project}_{algorithm.lower()}_{idx}'

In [ ]:
print(f'[{predictor_name_deep_ar}] Creating predictor {predictor_name_deep_ar} ...')

In [ ]:
create_predictor_response = forecast.create_predictor(PredictorName=predictor_name_deep_ar,
                                                  AlgorithmArn=algorithm_arn_deep_ar_plus,
                                                  ForecastHorizon=forecast_horizon,
                                                  PerformAutoML=False,
                                                  PerformHPO=False,
                                                  InputDataConfig= {"DatasetGroupArn": dataset_group_arn},
                                                  FeaturizationConfig= {"ForecastFrequency": freq}
                                                 )

In [ ]:
predictor_arn_deep_ar = create_predictor_response['PredictorArn']

In [ ]:
status = util.wait(lambda: forecast.describe_predictor(PredictorArn=predictor_arn_deep_ar))
assert status

In [ ]:
forecast.describe_predictor(PredictorArn=predictor_arn_deep_ar)

## Step 3b.  Choosing Prophet<a class="anchor" id="prophet">

In [ ]:
algorithm = 'Prophet'
algorithm_arn_prophet = algorithm_arn + algorithm
predictor_name_prophet = f'{project}_{algorithm.lower()}_{idx}'

In [ ]:
algorithm_arn_prophet

In [ ]:
print(f'[{predictor_name_prophet}] Creating predictor %s ...' % predictor_name_prophet)

In [ ]:
create_predictor_response = forecast.create_predictor(PredictorName=predictor_name_prophet,
                                                  AlgorithmArn=algorithm_arn_prophet,
                                                  ForecastHorizon=forecast_horizon,
                                                  PerformAutoML=False,
                                                  PerformHPO=False,
                                                  InputDataConfig= {"DatasetGroupArn": dataset_group_arn},
                                                  FeaturizationConfig= {"ForecastFrequency": freq}
                                                 )

In [ ]:
predictor_arn_prophet = create_predictor_response['PredictorArn']

In [ ]:
status = util.wait(lambda: forecast.describe_predictor(PredictorArn=predictor_arn_prophet))
assert status

In [ ]:
forecast.describe_predictor(PredictorArn=predictor_arn_prophet)

# Step 4. Computing Error Metrics from Backtesting<a class="anchor" id="error">

After creating the predictors, we can query the forecast accuracy given by the backtest scenario and have a quantitative understanding of the performance of the algorithm. Such a process is iterative in nature during model development. When an algorithm with satisfying performance is found, the customer can deploy the predictor into a production environment, and query the forecasts for a particular item to make business decisions. The figure below shows a sample plot of different quantile forecasts of a predictor.

In [ ]:
print('Done creating predictor. Getting accuracy numbers for DeepAR+ ...')

In [ ]:
error_metrics_deep_ar_plus = forecast.get_accuracy_metrics(PredictorArn=predictor_arn_deep_ar)
error_metrics_deep_ar_plus

In [ ]:
print('Done creating predictor. Getting accuracy numbers for Prophet ...')

In [ ]:
error_metrics_prophet = forecast.get_accuracy_metrics(PredictorArn=predictor_arn_prophet)
error_metrics_prophet

In [ ]:
def extract_summary_metrics(metric_response, predictor_name):
    df = pd.DataFrame(metric_response['PredictorEvaluationResults']
                 [0]['TestWindows'][0]['Metrics']['WeightedQuantileLosses'])
    df['Predictor'] = predictor_name
    return df

In [ ]:
deep_ar_metrics = extract_summary_metrics(error_metrics_deep_ar_plus, "DeepAR")
prophet_metrics = extract_summary_metrics(error_metrics_prophet, "Prophet")

In [ ]:
pd.concat([deep_ar_metrics, prophet_metrics]) \
    .pivot(index='Quantile', columns='Predictor', values='LossValue').plot.bar();

As we mentioned before, if you only have a handful of time series (in this case, only 1) with a small number of examples, the neural network models (DeepAR+) are not the best choice. Here, we clearly see that DeepAR+ behaves worse than Prophet in the case of a single time series. 

# Step 5. Creating a Forecast<a class="anchor" id="forecast">

Next we re-train with the full dataset, and create the forecast.

In [ ]:
print(f"Done fetching accuracy numbers. Creating forecaster for DeepAR+ ...")

In [ ]:
forecast_name_deep_ar = f'{project}_deep_ar_plus_{idx}'

In [ ]:
create_forecast_response_deep_ar = forecast.create_forecast(ForecastName=forecast_name_deep_ar,
                                                        PredictorArn=predictor_arn_deep_ar)

In [ ]:
forecast_arn_deep_ar = create_forecast_response_deep_ar['ForecastArn']

In [ ]:
status = util.wait(lambda: forecast.describe_forecast(ForecastArn=forecast_arn_deep_ar))
assert status

In [ ]:
forecast.describe_forecast(ForecastArn=forecast_arn_deep_ar)

In [ ]:
print(f"Done fetching accuracy numbers. Creating forecaster for Prophet ...")

In [ ]:
forecast_name_prophet = f'{project}_prophet_{idx}'

In [ ]:
create_forecast_response_prophet = forecast.create_forecast(ForecastName=forecast_name_prophet,
                                                        PredictorArn=predictor_arn_prophet)

In [ ]:
forecast_arn_prophet = create_forecast_response_prophet['ForecastArn']

In [ ]:
status = util.wait(lambda: forecast.describe_forecast(ForecastArn=forecast_arn_prophet))
assert status

In [ ]:
forecast.describe_forecast(ForecastArn=forecast_arn_prophet)

# Step 6. Querying the Forecasts<a class="anchor" id="query">

In [ ]:
item_id = 'bike_12'

In [ ]:
forecast_response_deep = forecast_query.query_forecast(
    ForecastArn=forecast_arn_deep_ar,
    Filters={"item_id": item_id})

In [ ]:
forecast_response_prophet = forecast_query.query_forecast(ForecastArn=forecast_arn_prophet,
                                                     Filters={"item_id":item_id})

In [ ]:
fname = f'data/bike_small.csv'
exact = util.load_exact_sol(fname, item_id)

In [ ]:
util.plot_forecasts(forecast_response_deep, exact)
plt.title("DeepAR Forecast");

In [ ]:
util.plot_forecasts(forecast_response_prophet,exact)
plt.title("Prophet Forecast");

# Step 7. Exporting your Forecasts<a class="anchor" id="export">

In [ ]:
forecast_export_name_deep_ar = f'{project}_forecast_export_deep_ar_plus_{idx}'
forecast_export_name_deep_ar_path = f"{s3_data_path}/{forecast_export_name_deep_ar}"

In [ ]:
create_forecast_export_response_deep_ar = forecast.create_forecast_export_job(ForecastExportJobName=forecast_export_name_deep_ar,
                                                        ForecastArn=forecast_arn_deep_ar,
                                                        Destination={
                                                            "S3Config" : {
                                                                "Path": forecast_export_name_deep_ar_path,
                                                                "RoleArn": role_arn
                                                            }
                                                        })
forecast_export_arn_deep_ar = create_forecast_export_response_deep_ar['ForecastExportJobArn']

In [ ]:
status = util.wait(lambda: forecast.describe_forecast_export_job(ForecastExportJobArn = forecast_export_arn_deep_ar))
assert status

In [ ]:
forecast_export_name_prophet = f'{project}_forecast_export_prophet_{idx}'
forecast_export_name_prophet_path = f"{s3_data_path}/{forecast_export_name_prophet}"

In [ ]:
create_forecast_export_response_prophet = forecast.create_forecast_export_job(ForecastExportJobName=forecast_export_name_prophet,
                                                        ForecastArn=forecast_arn_prophet,
                                                        Destination={
                                                            "S3Config" : {
                                                                "Path": forecast_export_name_prophet_path,
                                                                "RoleArn": role_arn
                                                            }
                                                        })
forecast_export_arn_prophet = create_forecast_export_response_prophet['ForecastExportJobArn']

In [ ]:
status = util.wait(lambda: forecast.describe_forecast_export_job(ForecastExportJobArn = forecast_export_arn_prophet))
assert status

# Step 8. Cleaning up your Resources<a class="anchor" id="cleanup">

Once we have completed the above steps, we can start to cleanup the resources we created. All delete jobs, except for `delete_dataset_group` are asynchronous, so we have added the helpful `wait_till_delete` function. 
Resource Limits documented <a href="https://docs.aws.amazon.com/forecast/latest/dg/limits.html">here</a>. 

In [ ]:
# Delete forecast export for both algorithms
util.wait_till_delete(lambda: forecast.delete_forecast_export_job(ForecastExportJobArn = forecast_export_arn_deep_ar))
util.wait_till_delete(lambda: forecast.delete_forecast_export_job(ForecastExportJobArn = forecast_export_arn_prophet))

In [ ]:
# Delete forecast for both algorithms
util.wait_till_delete(lambda: forecast.delete_forecast(ForecastArn = forecast_arn_deep_ar))
util.wait_till_delete(lambda: forecast.delete_forecast(ForecastArn = forecast_arn_prophet))

In [ ]:
# Delete predictor for both algorithms
util.wait_till_delete(lambda: forecast.delete_predictor(PredictorArn = predictor_arn_deep_ar))
util.wait_till_delete(lambda: forecast.delete_predictor(PredictorArn = predictor_arn_prophet))

In [ ]:
# Delete the target time series and related time series dataset import jobs
util.wait_till_delete(lambda: forecast.delete_dataset_import_job(DatasetImportJobArn=ts_dataset_import_job_arn))
util.wait_till_delete(lambda: forecast.delete_dataset_import_job(DatasetImportJobArn=rts_dataset_import_job_arn))

In [ ]:
# Delete the target time series and related time series datasets
util.wait_till_delete(lambda: forecast.delete_dataset(DatasetArn=ts_dataset_arn))
util.wait_till_delete(lambda: forecast.delete_dataset(DatasetArn=rts_dataset_arn))

In [ ]:
# Delete dataset group
forecast.delete_dataset_group(DatasetGroupArn=dataset_group_arn)

In [ ]:
# Delete IAM role
util.delete_iam_role( role_name )